In [3]:
from config import EPISODE_LEN
import numpy as np

def evaluate_constant_wind_model(env, model, model_name):
    obs, info = env.reset()
    print(obs)
    done = False

    degree_step = 3
    evaluations = 360 // degree_step
    total_rewards = np.zeros((evaluations, EPISODE_LEN))
    total_powers = np.zeros((evaluations, EPISODE_LEN))
    _, info = env.reset()
    for i in range(evaluations):
        for j in range(EPISODE_LEN):
            action, _states = model.predict(obs)
            if i == 0 and j < 5:
                #env.render()
                print(obs, len(obs))
            obs, reward, terminated, truncated, info = env.step(action)
            total_rewards[i, j] = reward
            total_powers[i, j] = info["power_output"]
            if terminated or truncated:
                break
        for _ in range(degree_step):
            obs, info = env.reset()
    # Save the rewards
    np.save(f"data/results/{model_name}_rewards.npy", total_rewards)
    np.save(f"data/results/{model_name}_powers.npy", total_powers)

In [ ]:
from env_utils import get_4wt_symmetric_env
from stable_baselines3 import PPO

env = get_4wt_symmetric_env(episode_length=EPISODE_LEN, privileged=False, sorted_wind=True, load_pyglet_visualization=True)
model = PPO.load("models/4wt_symmetric_model")

evaluate_constant_wind_model(env, model, "constant_wind_model")
